## Tensorflow 환경 세팅


Tensorflow lite micro는 현재 분리되어 없어진 기능이기에 특정 버전으로 되돌려야 합니다.


In [1]:
import os

main_path = os.getcwd()

# tensorflow 레포지토리를 복사해온다.
!git clone https://github.com/tensorflow/tensorflow.git

# tensorflow 레포지토리로 이동.
os.chdir('tensorflow')
os.getcwd()

Cloning into 'tensorflow'...
remote: Enumerating objects: 1863373, done.
remote: Counting objects: 100% (1177/1177), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 1863373 (delta 688), reused 943 (delta 570), pack-reused 1862196 (from 1)
Receiving objects: 100% (1863373/1863373), 1.01 GiB | 19.47 MiB/s, done.
Resolving deltas: 100% (1527949/1527949), done.
Updating files: 100% (33392/33392), done.


'/Users/jaewone/Downloads/tensorflow'

In [2]:
# tensorflow lite 분리 이전 버전으로 되돌립니다.
!git reset --hard 6491886

Updating files: 100% (31533/31533), done.
HEAD is now at 64918868e21 Merge pull request #52872 from tensorflow-jenkins/relnotes-2.4.4-12739


## Get Model


모델을 가져옵니다. 간단하게 magic_wand 예제를 사용하여 모델을 불러오는 것을 보이겠습니다.


In [7]:
import tarfile

magic_wand_project_dir = os.path.join(main_path, 'tensorflow/tensorflow/lite/micro/examples/magic_wand')

# examples/magic_wand/train 경로로 이동해서
os.chdir(os.path.join(magic_wand_project_dir, 'train'))

# 모델의 가중치 값을 받아옵니다.
!wget http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
data_tar_path = os.path.join(magic_wand_project_dir, 'train/data.tar.gz')

# 압축 해제
with tarfile.open(data_tar_path, 'r:gz') as file:
    file.extractall()

# 데이터를 준비하는 data_prepare.py 파일을 실행
!python data_prepare.py

# Person에 따라 데이터를 분할
!python data_split_person.py

--2024-09-14 15:45:37--  http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
download.tensorflow.org (download.tensorflow.org) 해석 중... 34.64.4.59, 34.64.4.27, 34.64.4.123, ...
다음으로 연결 중: download.tensorflow.org (download.tensorflow.org)|34.64.4.59|:80... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 451862 (441K) [application/x-gzip]
저장 위치: `data.tar.gz.1'

data.tar.gz.1       100%[===================>] 441.27K  --.-KB/s    /  0.04s   

2024-09-14 15:45:37 (10.7 MB/s) - `data.tar.gz.1' 저장함 [451862/451862]

data_length: 981
data_length:981
train_length:653
valid_length:136
test_length:192


In [8]:
# 모델 학습
!python train.py --model CNN --person true

Start to load data...
train_data_length:16325
valid_data_length:136
test_data_length:192
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

2024-09-14 15:46:21.578408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-14 15:46:21.578562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Start to build net...
Built CNN.
Start training...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 3, 8)         104       
                                                    

## C 소스 파일 생성


아두이노에 Model을 업로드하기 위해 C 소스 파일을 생성해야 합니다. 이를 위해 xxd 라이브러리가 필요합니다.

주의할 점은 linux와 mac 환경에서는 단순히 xxd를 설치, 혹은 이미 설치되어 있는 경우가 있지만, Windows 환경에서는 직접적으로 수행되지 않습니다. 추가적인 처리가 더 필요할 수도 있습니다.


In [13]:
# C source file를 만들기 위한 xxd 라이브러리 설치
!apt-get -qq install xxd

# 모델을 C 소스 파일로 변경
!xxd -i model.tflite > /model.cc

# C 소스 파일을 출력
!cat /model.cc

unsigned char model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x88, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00, 0x7c, 0x45, 0x00, 0x00,
  0x8c, 0x45, 0x00, 0x00, 0xe8, 0x4f, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x7a, 0xb9, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x38, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f,
  0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x94, 0xff, 0xff, 0xff, 0x11, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x64, 0x65, 0x6e, 0x73,
  0x65, 0x5f, 0x31, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x5a, 0xba, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x6

## 주의사항


Tensorflow 모델을 한번에 C 소스 파일로 변환할 수 없습니다. Tensorflow 모델을 tflite로 변환한 후, tflite 모델을 C 소스 파일로 변환해야 합니다.

본 프로젝트에서 사용한 모델은 [7.실제 데이터 분석 및 CNN 모델 생성/model.ipynb](https://github.com/jaewonE/define_floor_noise/blob/main/7.%20%EC%8B%A4%EC%A0%9C%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EB%B0%8F%20CNN%20%EB%AA%A8%EB%8D%B8%20%EC%83%9D%EC%84%B1/model.ipynb) 에서 확인하실 수 있습니다.
